# Mapeo de datos AirBnb Santiago

## Los datos fueron obtenidos desde http://insideairbnb.com/santiago/, para cualquier tipo de chequeo de datos

### Este pequeño proyecto de Jupyter contempla las siguientes etapas

* Importar las librerías
* Leer los datos desde el Excel obtenido
* Preparar los datos para trabajar con ellos
> Principalmente se trabajará con la latitud, la longitud, el precio (variable price) y la url (listing_url)
* Confeccionar el mapa

#### La librería gmaps es la librería de Google Maps, con la cual se trabajará en esta oportunidad

In [1]:
import pandas as pd
import csv
import gmaps
import gmaps.datasets
import numpy as np

#### Con la siguiente librería e instrucción, evitaremos que los warnings aparezcan

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv ("listings.csv")

In [4]:
lugares = df[["latitude", "longitude", "price", "listing_url"]]

#### En este trozo de código, se realiza una transformación de la variable price, limpiandola y transformándola a numérica

In [5]:
lugares["price"] = lugares["price"].str.replace("$", "")
lugares["price"] = lugares["price"].str.replace(".00", "")
lugares["price"] = lugares["price"].str.replace(",", "")
lugares["price"] = pd.to_numeric (lugares["price"], errors="coerce")

#### Con esta instrucción, se indica cual es la API de Google Maps para poder trabajar

In [6]:
gmaps.configure(api_key="AI..")

#### Se registraron valores extraños, como una propiedad con un precio de 180 millones, lo cual se chequeo en la página web de airbnb y el valor bordeaba los 65.000.
#### Para efectos de este notebook, solo se considerarán los departamentos con un valor de hasta $500.000

In [7]:
filtro = lugares[lugares["price"] <= 500000][["latitude", "longitude", "price", "listing_url"]]
precio_maximo = filtro["price"].max()

#### Con estas instrucciones, se genera el mapa de las propiedades para renta en AirBnb, generando un mapa de calor de acuerdo al precio, en donde la intensidad máxima de calor será la calculada previamente en la variable precio_maximo

In [9]:
fig = gmaps.figure(map_type="HYBRID")
mas_caros = gmaps.marker_layer(filtro[filtro["price"] > 300000][["latitude", "longitude"]],
            info_box_content='<a href='+filtro[filtro["price"] > 300000]["listing_url"].values+'>Detalle</a>')
fig.add_layer(gmaps.heatmap_layer(filtro[["latitude", "longitude"]], 
                                  weights=filtro["price"], max_intensity=500000))
fig.add_layer(mas_caros)
fig

Figure(layout=FigureLayout(height='420px'))